In [1]:
from tkinter import * # For the graphical interface
from tkinter import filedialog # So we can upload pictures
from pyzbar.pyzbar import decode #to decode the data in the barcode
from PIL import Image, ImageTk #Image Processing
import pandas as pd # To read the data set
import cv2 #For the camera
import re #We use it to extract numbers only from barcode data

excel_file_path = r"C:\Users\m7md7\Desktop\CV\Dataset\Serial code_dataset.xlsx"
data = pd.read_excel(excel_file_path) #Specify the path to the data set and the Panda library will read it 

class MedicineExplorerApp:#We created a Class and named it Medicine Explorer App, For variables and interface
    def __init__(self, master):
        self.master = master
        self.master.title("Medicine Explorer")
        self.master.geometry("1000x700")

        self.label = Label(master, text="Medicine Explorer", font=("Helvetica", 14))
        self.label.pack(pady=10)

        self.frame = Frame(master)
        self.frame.pack()

        self.camera_feed_dimensions = (600, 360)

        self.canvas = Canvas(self.frame, width=self.camera_feed_dimensions[0], height=self.camera_feed_dimensions[1])
        self.canvas.grid(row=0, column=0, padx=10, pady=10)

        self.product_info_text = Text(self.frame, wrap=WORD, font=("Helvetica", 14), height=15, width=55)
        self.product_info_text.grid(row=0, column=1, padx=10, pady=10)

        self.scan_button = Button(master, text="Start Scanning", command=self.scan_barcode)
        self.scan_button.pack(pady=10)

        self.upload_button = Button(master, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(pady=10)

        self.rescan_button = Button(master, text="Rescan", command=self.rescan)
        self.rescan_button.pack(pady=10)

        self.close_button = Button(master, text="Close", command=self.close_app)
        self.close_button.pack(pady=10)

        self.tk_image = None  # Placeholder for the PhotoImage

        self.cap = cv2.VideoCapture(0)
        self.cap.set(3, self.camera_feed_dimensions[0])
        self.cap.set(4, self.camera_feed_dimensions[1])

        self.delay = 1
        self.stopped = False
        self.close_requested = False

    def scan_barcode(self): ###It scans barcodes using the camera and displays the results in the user interface
        while True:
            ret, frame = self.cap.read()

            if not ret or self.stopped or frame is None:
                break

            decoded_objects = decode(frame)

            if decoded_objects is not None:
                for d in decoded_objects:
                    try:
                        barcode_value = int(re.search(r'\d+', d.data.decode('utf-8')).group())
                        print(barcode_value)

                        self.stopped = True

                        drug_data = data.loc[data["Serial num"] == barcode_value]
                        product_info = self.format_product_info(drug_data)
                        self.product_info_text.delete(1.0, END)
                        self.product_info_text.insert(END, product_info)

                        # Comment the line below to release the camera after scanning
                        self.cap.release()

                        return

                    except ValueError:
                        print("Invalid barcode value")
                        self.product_info_text.delete(1.0, END)
                        self.product_info_text.insert(END, "Invalid barcode value")

            if not self.stopped:
                cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_image = Image.fromarray(cv2image)
                self.tk_image = ImageTk.PhotoImage(image=pil_image)

                self.canvas.config(width=self.tk_image.width(), height=self.tk_image.height())
                self.canvas.create_image(0, 0, anchor=NW, image=self.tk_image)

            if cv2.waitKey(self.delay) & 0xFF == ord('q') or self.close_requested:
                break

            # Break out of the loop if the window is closed
            if not self.master.winfo_exists():
                break

            self.master.update()

        # Release the camera outside the loop
        self.cap.release()
        # Destroy the window when scanning is complete
        self.master.destroy()

    def close_app(self): # These functions close the application
        self.stopped = True
        self.cap.release()
        self.master.destroy()

    def rescan(self):# These functions rescan the application
        self.stopped = False
        self.product_info_text.delete(1.0, END)
        self.cap.release()
        self.cap = cv2.VideoCapture(0)
        self.cap.set(3, self.camera_feed_dimensions[0])
        self.cap.set(4, self.camera_feed_dimensions[1])

    def upload_image(self):# These functions upload the image  the application
        # Release the camera if it is open
        if not self.stopped:
            self.cap.release()

        file_path = filedialog.askopenfilename(title="Select Image File", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])

        if file_path:
            self.process_image(file_path)

    def process_image(self, image_path):# It processes the selected image and displays the barcode results in the user interface
        image = Image.open(image_path)
        decoded_objects = decode(image)

        for obj in decoded_objects:
            try:
                barcode_value = int(re.search(r'\d+', obj.data.decode('utf-8')).group())
                print(barcode_value)

                drug_data = data.loc[data["Serial num"] == barcode_value]
                product_info = self.format_product_info(drug_data)
                self.product_info_text.delete(1.0, END)
                self.product_info_text.insert(END, product_info)

            except ValueError:
                print("Invalid barcode value")
                self.product_info_text.delete(1.0, END)
                self.product_info_text.insert(END, "Invalid barcode value")

        image.close()

        img = Image.open(image_path)
        img = img.resize(self.camera_feed_dimensions, Image.LANCZOS)
        self.tk_image = ImageTk.PhotoImage(img)

        self.canvas.config(width=self.tk_image.width(), height=self.tk_image.height())
        self.canvas.create_image(0, 0, anchor=NW, image=self.tk_image)
        self.master.update()

    def format_product_info(self, drug_data):# Formats the information of the found product using the barcode
        if not drug_data.empty:
            product_info = f"Serial num: {drug_data['Serial num'].values[0]}\n"
            product_info += f"Product name: {drug_data['Product name'].values[0]}\n"
            product_info += f"Salt composition: {drug_data['Salt composition'].values[0]}\n"
            product_info += f"Reasons for use: {drug_data['Reasons for use'].values[0]}\n"
            product_info += f"Recommended dose: {drug_data['Recommended dose'].values[0]}\n"
            product_info += f"Side effects: {drug_data['Side effects'].values[0]}\n"
            product_info += f"Drug interactions: {drug_data['Drug interactions'].values[0]}"
        else:
            product_info = "Product not found in the database."

        return product_info

def main():# This function creates a Tkinter window and runs the application.
    root = Tk()
    app = MedicineExplorerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
